In [27]:
%run data_processor.py

clean the text: 10662it [00:00, 30408.74it/s]
10662it [00:00, 30213.62it/s]

Average length: 20.38585631213656
doc count: 10662
Total number of words: 18764


In [28]:
%run build_graph.py


==> 现在的数据集是:mr<==


generate tfidf edge: 1985it [00:00, 9839.89it/s]

tfidf time: 0.3178136348724365
tfidf_vec shape: (10662, 18764)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 18764


generate tfidf edge: 10662it [00:01, 8450.67it/s]
Split by window:   3%|██                                                         | 363/10662 [00:00<00:02, 3594.15it/s]

+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  196826  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
pmi read file len: 10662


Calculate pmi between words: 100%|███████████████████████████████████████| 1038228/1038228 [00:03<00:00, 321818.45it/s]


Total number of edges between word: 788864
pmi time: 6.823136329650879
+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  918907  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
total time: 7.140949964523315




In [26]:
# target_fn = "../tmp/TCGCN/graph/mr.txt"
target_fn = "data/mr.txt"
target = np.array(pd.read_csv(target_fn,sep="\t", header=None, engine='python')[2])
target2id = {label: indx for indx, label in enumerate(set(target))}
target = [target2id[label] for label in target]
nclass = len(target2id)
len(target), nclass

(10662, 2)

In [29]:
%%time
%run trainer.py

Namespace(data_path='./data', dataset='mr', device=device(type='cpu'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, temp_path='../tmp/TCGCN', use_gf=False, val_ratio=0.1)
prepare data
+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  918907  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+


==> 0, seed:73662
<bound method Module.parameters of GCN(
  (gc1): GraphConvolution (29426 -> 200)
  (gc2): GraphConvolution (200 -> 2)
)>
# model parameters: 5885802
epoch:0 train_loss:0.6934 val_loss:0.6776 acc:0.5162 macro_f1:0.6035 precision:0.7577 recall:0.5014 
epoch:1 train_loss:0.6625 val_loss:0.6529 acc:0.5851 macro_f1:0.6602 precision:0.7390 recall:0.5966 
epoch:2 train_loss:0.6150 val_loss:0.6297 acc:0.6582 macro_f1:0.7119 precision:0.7628 reca

In [31]:
%%time
%run trainer.py

Namespace(data_path='./data', dataset='mr', device=device(type='cpu'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, temp_path='../tmp/TCGCN', use_gf=True, val_ratio=0.1)
prepare data
+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  918907  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+


==> 0, seed:94451
<bound method Module.parameters of GCN(
  (gc1): GraphConvolution (29426 -> 200)
  (gc2): GraphConvolution (200 -> 2)
)>
# model parameters: 5885802
epoch:0 train_loss:0.7476 val_loss:0.7117 acc:0.5316 macro_f1:0.5716 precision:0.6084 recall:0.5390 
epoch:1 train_loss:0.6981 val_loss:0.6993 acc:0.5373 macro_f1:0.5831 precision:0.6274 recall:0.5447 
epoch:2 train_loss:0.6748 val_loss:0.6667 acc:0.5935 macro_f1:0.6189 precision:0.6406 recal

In [9]:
%run data_processor.py

clean the text: 10662it [00:00, 23332.69it/s]
10662it [00:00, 19997.43it/s]

Average length: 20.38585631213656
doc count: 10662
Total number of words: 18764


In [1]:
%run build_graph.py


==> 现在的数据集是:mr<==


generate tfidf edge: 1032it [00:00, 10315.31it/s]

tfidf time: 0.2519819736480713
tfidf_vec shape: (10662, 18764)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 18764


generate tfidf edge: 10662it [00:01, 8840.82it/s]
Split by window:   3%|██                                                         | 363/10662 [00:00<00:02, 3594.57it/s]

+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  196826  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
pmi read file len: 10662


Calculate pmi between words: 100%|███████████████████████████████████████| 1043618/1043618 [00:02<00:00, 520498.23it/s]


Total number of edges between word: 790158
pmi time: 6.098142623901367
+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  918137  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
total time: 6.3501245975494385




In [5]:
def b_c(G):
    vertices = G.nodes()
    new_bc = {}
    paths = defaultdict(dict)

    # Get shortest paths between all pairs of vertices
    for i, vertex in enumerate(vertices[:-1]):
        for o_vertex in vertices[i+1:]:
            paths[vertex][o_vertex] = [path for path in
                                       nx.all_shortest_paths(G, vertex, o_vertex)]

    for vertex in vertices:
        counter = 0
        for i, vertex1 in enumerate(vertices[:-1]):
            for vertex2 in vertices[i+1:]:
                for path in paths[vertex1][vertex2]:
                    if vertex in path[1:-1]:
                        counter += 1
        new_bc[vertex] = counter

    return new_bc

In [21]:
vertices = G.g.nodes()
len(vertices), vertices
# for i, vertex in enumerate(vertices[:-1]):
#     print(i)

(29426, {})

In [32]:
import torch as th
import numpy as np
value = [10,20,30]#[1.] * 3
shape = (3, 3)
indices = th.from_numpy(
        np.vstack((list(range(3)), list(range(3)))).astype(np.int64))
values = th.FloatTensor(value)
shape = th.Size(shape)
features = th.sparse.FloatTensor(indices, values, shape)
features

tensor(indices=tensor([[0, 1, 2],
                       [0, 1, 2]]),
       values=tensor([10., 20., 30.]),
       size=(3, 3), nnz=3, layout=torch.sparse_coo)

In [ ]:
from networkx.algorithms.centrality import eigenvector_centrality
ec = eigenvector_centrality(G.g)

# موارد زیر جواب ندادن
# closeness_centrality
#betweenness_centrality
# bc = betweenness_centrality(G.g, k =1000)

# bc = b_c(G.g)

In [67]:
# len(ec), ec[1]
# type(ec)
# ec.value
# Creates a sorted dictionary (sorted by key)
from collections import OrderedDict

# dict = {'ravi':'10','rajnish':'9','sanjeev':'15','yash':'2','suraj':'32'}
dict1 = OrderedDict(sorted(ec.items()))
# print(dict1)
# ec_list = dict1.values()
ec_list = [x for x in dict1.values()]
# list(ec_list)
np.min(ec_list), np.max(ec_list), np.mean(ec_list)

(1.4121735301786423e-72, 0.14859581062595026, 0.00373242862585294)

In [68]:
ec_list[:4]

[0.003201180323761659,
 0.00041326022033528353,
 0.0018442263122221427,
 0.0023540925747042147]

In [25]:
# len(G.g.nodes())
G.g[0]

AtlasView({27573: {'weight': 5.855600689586108}, 26169: {'weight': 5.855600689586108}, 25806: {'weight': 10.274441297382706}, 25548: {'weight': 8.888146936262816}, 23636: {'weight': 10.274441297382706}, 23559: {'weight': 10.274441297382706}, 22465: {'weight': 5.070434610305911}, 22149: {'weight': 1.8349934545913218}, 20475: {'weight': 3.7119972036889863}, 20428: {'weight': 10.274441297382706}, 19699: {'weight': 2.240134361043218}, 19590: {'weight': 4.2582841376843525}, 19538: {'weight': 6.304149383830585}, 19178: {'weight': 2.5195310253612764}, 18998: {'weight': 6.34261566465838}, 18685: {'weight': 3.9718223216378012}, 18493: {'weight': 4.654040431665557}, 18225: {'weight': 8.665003384948605}, 14635: {'weight': 9.175829008714597}, 13717: {'weight': 8.077216720046486}, 13170: {'weight': 8.48268182815465}, 12994: {'weight': 8.19499975570287}, 12008: {'weight': 3.1229558334779712}, 11720: {'weight': 1.7656835847875705}, 11141: {'weight': 5.029217154367105}, 10954: {'weight': 1.54650107498

In [ ]:
pos = nx.nx_pydot.pydot_layout(G.g)
plt.figure()    
nx.draw(G.g,pos,edge_color='black',width=1,linewidths=1, node_size=10,node_color='blue',alpha=0.9)
plt.axis('on')
plt.show()

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 

fig = plt.figure(figsize=(40, 40)) 
# G = nx.fast_gnp_random_graph(300, 0.02, seed=1337) 
nx.draw(G.g, node_size=30) 
plt.axis('equal') 
plt.show() 
fig.savefig('waka.svg')